# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si; psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846792703375                   -0.70    4.5         
  2   -7.852309245713       -2.26       -1.53    1.0   16.5ms
  3   -7.852616188595       -3.51       -2.56    1.5   18.5ms
  4   -7.852646009648       -4.53       -2.89    2.5   22.2ms
  5   -7.852646518176       -6.29       -3.19    1.2   17.5ms
  6   -7.852646679836       -6.79       -4.13    1.0   16.8ms
  7   -7.852646686598       -8.17       -5.18    2.2   21.7ms
  8   -7.852646686726       -9.89       -5.57    1.8   20.4ms
  9   -7.852646686725   +  -12.03       -5.50    1.5   19.4ms
 10   -7.852646686730      -11.37       -6.23    1.0   17.1ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846878054357                   -0.70           4.8         
  2   -7.852525959879       -2.25       -1.64   0.80    2.0    205ms
  3   -7.852634993341       -3.96       -2.73   0.80    1.0   15.6ms
  4   -7.852646448171       -4.94       -3.24   0.80    2.2   20.3ms
  5   -7.852646664001       -6.67       -4.04   0.80    1.0   15.6ms
  6   -7.852646686331       -7.65       -4.68   0.80    1.8   18.5ms
  7   -7.852646686704       -9.43       -5.42   0.80    1.5   17.4ms
  8   -7.852646686729      -10.61       -6.65   0.80    2.0   19.0ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.388680e+01     3.262586e+00
 * time: 0.04551386833190918
     1     1.454994e+00     1.833208e+00
 * time: 0.21845698356628418
     2    -1.680265e+00     1.929297e+00
 * time: 0.23627686500549316
     3    -4.043369e+00     1.663724e+00
 * time: 0.26180577278137207
     4    -5.419878e+00     1.606568e+00
 * time: 0.28751182556152344
     5    -7.074710e+00     8.729292e-01
 * time: 0.3132328987121582
     6    -7.246101e+00     1.026317e+00
 * time: 0.3310818672180176
     7    -7.667265e+00     6.404834e-01
 * time: 0.34896183013916016
     8    -7.746694e+00     4.700374e-01
 * time: 0.36675095558166504
     9    -7.782394e+00     1.154751e-01
 * time: 0.3846099376678467
    10    -7.811365e+00     1.061210e-01
 * time: 0.40245985984802246
    11    -7.831738e+00     5.789616e-02
 * time: 0.4203369617462158
    12    -7.840892e+00     5.156046e-02
 * time: 0.4381678104400635
    13    -7.846042e+00     4.980181e-02
 * time: 0.45

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846904062777                   -0.70    4.8         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645933826                   -1.64         
  2   -7.852646686730       -6.12       -3.71    1.46s
  3   -7.852646686730      -13.35       -7.26    116ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 7.090826344524038e-7
|ρ_newton - ρ_scfv| = 5.965486631022442e-7
|ρ_newton - ρ_dm|   = 1.6640504564903207e-9
